In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import time

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta = "C:/Users/cristian.guatemal/Documents/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [3]:
#Importación de archivos de afiliados
inicio = time.time()
afi = pd.read_csv( ruta_afi, delimiter='\t', encoding='iso-8859-1', decimal='.')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  1.0  minutos y 54.736570596694946  segundos


In [4]:
#Agrupamiento por relación de trabajo
inicio = time.time()
agrupa = [ (afi['SECTOR'].isin(['PRIVADO', 'PASANTE PRIVADO'])), 
               (afi['SECTOR'].isin(['PUBLICO', 'PASANTE PUBLICO'])), 
               (afi['SECTOR'].isin(['INDEPENDIENTES', '108-ARTISTA Y GESTOR DE CULTURA', 
                                     '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  '])),
               (afi['SECTOR'].isin(['69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
                                    '89-AFILIACION DOMICILIADO EN EL EXTERIOR'])),
               (afi['SECTOR'] == '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA'),
               (afi['SECTOR'] == '06-CODIGO DEL TRABAJO - CT ')
            ]
            
tipo = ['PRI', 'PUB', 'IND', 'VOL_EX', 'VOL_EC', 'COD_TR']

afi['SECTOR_A'] = np.select( agrupa, tipo, default=afi['SECTOR'])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', 
        (fin-inicio)%60, ' segundos' )

Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 16.630993127822876  segundos


In [5]:
afi['SECTOR'].unique()

array(['PRIVADO', 'PUBLICO',
       '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  ',
       'INDEPENDIENTES',
       '69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
       'PASANTE PRIVADO',
       '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA',
       '89-AFILIACION DOMICILIADO EN EL EXTERIOR', 'PASANTE PUBLICO',
       '108-ARTISTA Y GESTOR DE CULTURA', '06-CODIGO DEL TRABAJO - CT '],
      dtype=object)

In [6]:
afi['SECTOR_A'].unique()

array(['PRI', 'PUB', 'IND', 'VOL_EX', 'VOL_EC', 'COD_TR'], dtype=object)

In [7]:
afi.shape

(68062270, 13)

In [8]:
#Para acceder a las prestaciones de IVM debe tener al menos 5 años de aporte o 60 imposiciones
ced_counts = afi['CEDULA_COD'].value_counts()

In [9]:
# numero de cedulas que tienen menos de 60 imposiciones
ced_counts[ced_counts < 60].index.nunique()

112665

In [10]:
#Se filtran a las cedulas con menos de 60 imposicones 
inicio = time.time()
afi_fil = afi[~afi['CEDULA_COD'].isin(ced_counts[ced_counts < 60].index)].copy()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se crea una variable fecha
inicio = time.time()
afi_fil.loc[:, 'FECHA'] = pd.to_datetime(afi_fil['ANIO'].astype(str) + '-' + afi_fil['MES'].astype(str).str.zfill(2) + '-01')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

#Se ordenan los registros
inicio = time.time()
afi_fil = afi_fil.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, True, True] )
afi_fil.loc[:, 'FECHA'] = pd.to_datetime(afi_fil['ANIO'].astype(str) + '-' + afi_fil['MES'].astype(str).str.zfill(2) + '-01')
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
afi_fil.shape

Tiempo de ejecución es:  0.0  horas con  0.0  minutos y 28.073200225830078  segundos
Tiempo de ejecución es:  0.0  horas con  1.0  minutos y 17.89201283454895  segundos
Tiempo de ejecución es:  0.0  horas con  2.0  minutos y 57.444562673568726  segundos


(64756154, 14)

In [11]:
del afi
del ced_counts
afi_fil

,CEDULA_COD,ANIO,MES,CODSEC,SECTOR,DESTIPEMP,DESRELTRA,CODCII,PROVINCIA,CANTON,SALARIO,PORCENTAJE_AP,SECTOR_A,FECHA
26481585,126,2004,12,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,147.45,20.5,PRI,2004-12-01
67442511,126,2005,1,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,41.0,PRI,2005-01-01
41179435,126,2005,2,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,PRI,2005-02-01
44033860,126,2005,3,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,PRI,2005-03-01
47099991,126,2005,4,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,K66220001,AZUAY,CUENCA,161.67,20.5,PRI,2005-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27896256,23594465,2017,11,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,14045.72,20.6,PRI,2017-11-01
5107034,23594465,2017,12,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,14164.25,20.6,PRI,2017-12-01
44333802,23594465,2018,1,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,12903.21,20.6,PRI,2018-01-01
38102314,23594465,2018,2,R,PRIVADO,2-EMPRESA PRIVADA -SOCIEDADES / COMPANIAS,06-CODIGO DEL TRABAJO - CT,C111,PICHINCHA,DISTRITO METROPOLITANO DE QUITO,42432.21,20.6,PRI,2018-02-01


In [12]:
#Se seleccionan las características de la población para los año y mes de aportación
afi_sel = afi_fil[['CEDULA_COD', 'ANIO', 'MES', 'FECHA','SALARIO', 'SECTOR_A']].copy()
del afi_fil

In [13]:
#Inicio de la HL
caract = afi_sel.groupby('CEDULA_COD')['FECHA'].min().reset_index() 
caract.rename( columns={'FECHA': 'INI_HL'}, inplace=True)
#Fin de la HL
caract['FIN_HL'] = caract['CEDULA_COD'].map( afi_sel.groupby('CEDULA_COD')['FECHA'].max() )
#Contar los meses, considerando duplicidad en los meses que puede tener aporte simultaneos
caract['MES_C'] = caract['CEDULA_COD'].map( afi_sel['CEDULA_COD'].value_counts() )
#Meses trabajados unicos
aux = afi_sel.drop_duplicates(subset=['CEDULA_COD', 'ANIO', 'MES'])
caract['MES_T'] = caract['CEDULA_COD'].map( aux['CEDULA_COD'].value_counts() )

In [14]:
caract

,CEDULA_COD,INI_HL,FIN_HL,MES_C,MES_T
0,126,2004-12-01,2010-12-01,73,73
1,152,2004-07-01,2009-09-01,63,63
2,216,2004-07-01,2009-09-01,98,63
3,230,2004-12-01,2015-05-01,126,126
4,235,2004-08-01,2011-06-01,83,83
...,...,...,...,...,...
444112,23075971,2017-02-01,2022-10-01,66,66
444113,23402574,2010-01-01,2019-02-01,110,110
444114,23456125,2016-07-01,2022-03-01,69,69
444115,23503686,2004-09-01,2011-11-01,87,87


In [15]:
#Personas que tienen varios aportes simultaneos
caract[caract['MES_C']>caract['MES_T']]

,CEDULA_COD,INI_HL,FIN_HL,MES_C,MES_T
2,216,2004-07-01,2009-09-01,98,63
10,397,2004-07-01,2012-01-01,138,91
12,419,2004-11-01,2009-12-01,115,58
20,461,2004-07-01,2010-09-01,221,75
23,472,2004-09-01,2012-01-01,176,89
...,...,...,...,...,...
444102,20949683,2011-08-01,2019-07-01,92,91
444104,21035240,2010-03-01,2019-09-01,112,111
444105,21035907,2006-07-01,2013-08-01,68,67
444106,21104059,2014-07-01,2023-02-01,105,104


In [16]:
#afi_sel[afi_sel['CEDULA_COD']==21729781].tail(60)

In [17]:
afi_sel =  afi_sel.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, False, False] )
afi_sel['NUM_SEC_MES'] = 1
afi_sel['%_NUM_SECTOR'] = afi_sel['SALARIO']

In [18]:
aux = afi_sel[afi_sel.duplicated(subset=['CEDULA_COD', 'ANIO', 'MES'], keep=False)]
aux1 = afi_sel[~afi_sel.duplicated(subset=['CEDULA_COD', 'ANIO', 'MES'], keep=False)]

In [ ]:
#Se debe analizar a las personas que tienen varios aportes en un mismo mes
#Se suman los salarios y se concatenan los sectores para la cedula que en el mismo anio y mes tienen más de un sector

inicio = time.time()
# afi_sel_g = aux.groupby(['CEDULA_COD', 'ANIO', 'MES']).agg({'SALARIO': 'sum',
#                                                             'SECTOR_A': lambda x: ';'.join(x),
#                                                             'NUM_SEC_MES': lambda x: len(x),
#                                                             '%_NUM_SECTOR': lambda x:  ';'.join((x / x.sum()).round(4).astype(str))
#                                                                 }).reset_index()
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )


In [ ]:
afi_sel

In [ ]:
afi_sel_g_all = pd.concat([aux1[['CEDULA_COD', 'ANIO', 'MES', 'SALARIO', 'SECTOR_A', 'NUM_SEC_MES', '%_NUM_SECTOR']], 
                           afi_sel_g], ignore_index=True)

In [ ]:
afi_sel_g_all = afi_sel_g_all.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[True, False, False] )
afi_sel_g_all 

In [ ]:
afi_sel_g_all[afi_sel_g_all['NUM_SEC_MES']>1]

In [ ]:
afi_sel[afi_sel['CEDULA_COD']==216].head(60)

In [ ]:
del aux
del aux1

In [ ]:
import multiprocessing

# Función de ejemplo que realiza una tarea intensiva en CPU
def compute_square(data):
    afi_sel_g = data.groupby(['CEDULA_COD', 'ANIO', 'MES']).agg({'SALARIO': 'sum',
                                                            'SECTOR_A': lambda x: ';'.join(x),
                                                            'NUM_SEC_MES': lambda x: len(x),
                                                            '%_NUM_SECTOR': lambda x:  ';'.join((x / x.sum()).round(4).astype(str))
                                                                }).reset_index()
    return afi_sel_g

if __name__ == '__main__':
    # Número de núcleos disponibles
    num_cores = multiprocessing.cpu_count()
  
    # Datos de ejemplo
    data = aux
    
    # Dividir el DataFrame en partes iguales
    data_split = np.array_split(data, num_cores)
    
    # Crear un Pool de procesos
    pool = multiprocessing.Pool(processes=num_cores)

    # Medir el tiempo de ejecución
    start_time = time.time()

    # Dividir la tarea y ejecutarla en paralelo
    results = pool.map(compute_square, data_split)

    # Cerrar el Pool y esperar a que todos los procesos terminen
    pool.close()
    pool.join()
    
    # Combinar los resultados
    combined_results = pd.concat(results)
    
    # Medir el tiempo de ejecución
    end_time = time.time()

    print(f"Tiempo de ejecución en paralelo: {end_time - start_time} segundos")

In [ ]:
#Se crea la variable que cuenta el número de sectores para cada anio y mes
inicio = time.time()
afi_sel_g['NUM_SEC_MES'] = afi_sel.groupby(['CEDULA_COD', 'ANIO', 'MES'])['SECTOR'].nunique().values
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )


In [ ]:
inicio = time.time()
#Se suma por cedula, anio, mes y sector
temp_df = afi_sel.groupby(['CEDULA_COD', 'ANIO', 'MES', 'SECTOR']).agg({'SALARIO': 'sum'}).reset_index()
sum_df = temp_df.groupby(['CEDULA_COD', 'ANIO', 'MES'])['SALARIO'].sum().reset_index()
sum_df.rename(columns={'SAL': 'TOTAL_SAL'}, inplace=True)
temp_df = temp_df.merge( sum_df, on=['CEDULA_COD', 'ANIO', 'MES'])
fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
sum_df = temp_df.groupby(['CEDULA_COD', 'ANIO', 'MES'])['SALARIO'].sum().reset_index()

In [ ]:
temp_df

In [ ]:
sum_df

In [ ]:
inicio = time.time()
#Se calcula el ´porcentaje del sueldo en cada sector
temp_df['PERC'] = temp_df.groupby(['CEDULA_COD', 'ANIO', 'MES'])['SALARIO'].transform(lambda x: (x / x.sum()).round(4).astype(str))

# #Se concatenan los porcentajes de sueldo de cada sector
# perc_concat = temp_df.groupby(['CEDULA_COD', 'ANIO', 'MES'])['PERC'].apply(lambda x: ';'.join(x)).reset_index()

# #Se cambia de nombre a la variable
# perc_concat.rename(columns={'PERC': '%_NUM_SECTOR'}, inplace=True)

fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
afi_sel[afi_sel['CEDULA_COD']==216].head(60)

In [ ]:
#se crean los grupos de 12 meses trabajados para tener
n_grupo = 12
afi_sel['GRUPO'] = (afi_sel.groupby('CEDULA_COD').cumcount() // n_grupo ) + 1
#Se saca el salario promedio de cada grupo
afi_sel['SAL_PROM_GRUPO'] = afi_sel.groupby(['CEDULA_COD', 'GRUPO'])['SALARIO'].transform('mean')


In [ ]:
afi_sel[afi_sel['CEDULA_COD']==126].head(60)

In [ ]:
afi_sel.groupby(['CEDULA_COD', 'GRUPO'])['SAL_PROM_GRUPO'].unique()
#     df1 = df1.reset_index()
#     df1['SAL_PROM_GRUPO'] = df1['SAL_PROM_GRUPO'].apply(lambda x: x[0])
#     df1 = df1.sort_values(by = ['CEDULA_COD','SAL_PROM_GRUPO'], ascending=[True,False])

In [ ]:
afi_sel.groupby('CEDULA_COD')['FECHA'].min()
#grupo['ANIO_MIN'] = grupo['CEDULA_COD'].map( grupo.groupby('CEDULA_COD')['ANIO'].min() )
#    grupo['MES_MIN']  = grupo['CEDULA_COD'].map(grupo[ grupo['ANIO']==grupo['ANIO_MIN']].groupby('CEDULA_COD')['MES'].min())